# Homework 3: Web Security

### Course: Information Security

### **Names**
Ericson López

Pablo Herrera

## **Part 1: Research and Analysis of DDoS Attack Techniques (Unchanged)**

**Objective:** Students will conduct in-depth research on the common methods used to execute Distributed Denial of Service (DDoS) attacks, focusing on the layers of the OSI model they target and how they operate.

* Task A: Categorization and Description  
  Research and define the three primary categories of DDoS attacks (Volumetric, Protocol, Application Layer). For each category, describe its goal and provide at least two specific attack examples. Present your findings in a comparison table.

### Volumetric
Volumetric attacks seek to saturate all available bandwidth between the victim and the rest of the Internet. Their main objective is to “fill the pipe” with junk traffic so that legitimate packets cannot reach the server. They operate mainly in layers 3 (network) and 4 (transport) of the OSI model, generating enormous volumes of packets per second. An example of this type is NTP Amplification, which exploits misconfigured time servers (NTP): the attacker sends small UDP queries with the victim's spoofed IP, and the server responds with much larger packets addressed to that victim, generating a massive flow of traffic. Another example is SSDP Amplification, which uses vulnerable devices that implement UPnP/SSDP; by sending requests with spoofed IPs, the devices respond to the target, flooding its network with amplified traffic.

### Protocol
Protocol attacks, also known as state exhaustion attacks, do not always require large volumes of data; their goal is to exhaust the state tables and resources of devices such as firewalls, load balancers, and servers. They continue to act on layers 3 and 4 of the OSI model, but instead of just consuming bandwidth, they fill the memory structures where connections are recorded. One example is the TCP ACK Flood, where massive TCP packets with forged ACK flags are sent; the server must process them, which consumes CPU and other resources, preventing it from handling legitimate traffic. Another example is the IP Fragmentation attack (Teardrop/Fragmentation Flood), which exploits IP or UDP packet fragmentation to send malformed fragments or fragments designed to confuse reassembly at the destination or routers, which can collapse the network stack or cause failures.

### Application Layer
Application layer attacks target layer 7 of the OSI model, where protocols such as HTTP, DNS, and SMTP reside, and aim to exhaust the application's resources (web server threads, processes, database queries, etc.). Unlike volumetric attacks, they may use little bandwidth, but they mimic legitimate traffic very well, making them difficult to detect. One example is DNS Flood, where a large volume of DNS requests (not necessarily amplified) are sent to a name server in order to saturate it and cause failures or slowdowns in name resolution. Another modern example is an HTTP/2 Rapid Reset Attack, where HTTP/2 (or similar) features are abused to generate many canceled or incomplete requests, forcing the server to process them repeatedly and exhausting its resources, without the need for a high volume of data. 
| DDoS Category | OSI Layer(s) Targeted | Primary Goal of Attack | Specific Attack Example 1 | Specific Attack Example 2 |
| :---- | :---- | :---- | :---- | :---- |
| **Volumetric** | L3, L4 | Consuming all available bandwidth, saturating the network link | **NTP Amplification** | **SSDP Amplification** |
| **Protocol** | L3, L4 | Exhaust the status tables and resources of network devices and servers | **TCP ACK Flood** | **IP Fragmentation attack** |
| **Application Layer** | L7 | Exhaust application/server resources (CPU, memory, database) | **DNS Flood** | **HTTP/2 Rapid Reset Attack** |\

* Task B: In-Depth Attack Profile  
  Choose one specific attack from your table (e.g., DNS Amplification) and write a detailed profile (250-350 words) that explains its Mechanism, the Resource Exhaustion it causes, and how Botnets/Amplification are used.

### NTP amplification
The NTP amplification attack is a type of volumetric reflection and amplification DDoS attack that abuses the Network Time Protocol (NTP), used to synchronize time between systems. Its central idea is to send very small NTP requests to vulnerable servers, spoofing the victim's IP address, so that those servers respond with much larger responses directed at the target. In many cases, the historical monlist command is exploited, which returns a list of up to 600 recent clients; this can generate responses tens or even hundreds of times larger than the original request. 

#### Mechanism:
- The attacker (or their botnet) sends UDP packets to port 123 of multiple misconfigured NTP servers.
- Each packet carries the victim's IP as the source address (IP spoofing).
- The NTP server processes the request (e.g., monlist) and generates a very large response.

All of these responses are sent to the victim, who never actually made the request. 

#### Resource exhaustion:
The main resource affected is bandwidth: the link to/from the victim's network becomes saturated with unsolicited UDP traffic, preventing legitimate traffic (web, email, VPN, etc.) from passing through. In addition, the huge number of packets can overload routers, firewalls, and other intermediate equipment, causing congestion or even service outages upstream, not just on the target server. 

#### Botnets and amplification:
The danger of the attack skyrockets when it combines a botnet that sends many small requests from thousands of compromised computers, thousands of vulnerable NTP servers acting as “mirrors,” and an amplification factor that can exceed 100x in some scenarios.
Thus, a few megabits of request traffic can be transformed into hundreds of gigabits of response traffic against the victim, as seen in real attacks of over 400 Gbps, making NTP Amplification one of the most efficient variants of volumetric DDoS.

## **Part 2: Hands-On: DVWA Deployment and Common Attacks**

**Objective:** Students will deploy a vulnerable web application using Docker and execute common web attacks in a safe, controlled environment, learning to bypass **basic** security measures.

**⚠️ Ethical Warning:** This task is for **educational, local simulation only**. Students are **strictly forbidden** from attempting these attacks on any public, external, or third-party web resource.

* **Task A: DVWA Installation via Docker**  
  1. Ensure **Docker** is installed and running on your local machine.  
  2. Pull and run the Damn Vulnerable Web Application (DVWA) using a public Docker image (e.g., vulnerables/web-dvwa or citizenstig/dvwa). You will need to map a local port (e.g., port 80\) to the container's web server.  
     **Example Command:** docker run \--rm \-it \-p 80:80 vulnerables/web-dvwa  
  3. Access the DVWA login page at http://localhost/ (or your mapped port). Log in with the default credentials (admin/password) and complete the database setup.  
  4. **Submission:** Provide the exact Docker command(s) you used to pull and run the container, and a screenshot of the DVWA home page. 

### Commands
Pull the DVWA image explicitly

`docker pull vulnerables/web-dvwa`

Run DVWA in a container, exposing port 80 of the container to port 80 on your host

`docker run --rm -it -p 80:80 vulnerables/web-dvwa`

### Screenshot

![](./images/DVWA.png)

* **Task B: Attack Execution and Documentation (Medium Security)**  
  1. Set the DVWA security level to **Medium**. This level often includes functions like mysql\_real\_escape\_string() for SQLi, or basic string replacement for XSS, requiring students to look for weaknesses like lack of case sensitivity or incomplete filtering.  
  2. Successfully execute **three** distinct common web application attacks using the DVWA menu options. Focus on vulnerabilities from the **OWASP Top 10**:  
     * **SQL Injection (SQLi)**: You may need to use **time-based blind techniques** or other bypasses since the standard UNION attack may be sanitized.  
     * **Cross-Site Scripting (XSS)**: You may need to bypass basic input filters by using different encodings, event handlers, or mixed-case tags.  
     * **Command Injection**: Look for ways to bypass filtering of commands like | or & by using other command separators (e.g., &&, ||, or \\n).  
  3. For each of the three attacks, provide:  
     * The **Vulnerability Name** (e.g., SQL Injection).  
     * The **Payload** used (the malicious input string).  
     * A **Screenshot** of the successful exploitation (e.g., the alert box for XSS, or the SQL query result for SQLi).  
     * A **brief explanation** (1-2 sentences) of *why* the attack payload needed to be modified to bypass the Medium security filter.

### SQL Injection
#### Payload used
The system was vulnerated using Postman and HTTP method called when Sumbit button is clicked on SQLi DVWA page.
At first, session cookies and security cookies were setted, after that the malicious string and another body params were setted on the request.
The malitious strings was: 1 OR 1=1

![](./images/SQL-injection-string.png)

#### Successful Explotation

In response of the method a new HTTP page state of the HTML SQLi DVWA was returned. In this HTML is visible with Postman preview tool the succesful result of the SQLi attack.

![](./images/SQL-injection-No-WAF.png)

#### Brief Explanation 

The payload attack needed to be modified to bypass the Medium security filter cause in the previous security level all the info related to the request was setted in the header, and for this reason the SQLi attack can be done in the same browser used to navigate to the DVWA app. However, in medium security level the information related to the request is setted on the body of the request and for this reason a tool like Postman is needed to include the malicious string in the request.

### Cross-Site Scripting (XSS)
#### Payload used
```bash
<br>
<sCript>
(function(){
    const overlay = document.createElement("div");
    overlay.style.position = "fixed";
    overlay.style.top = "0";
    overlay.style.left = "0";
    overlay.style.width = "100%";
    overlay.style.height = "100%";
    overlay.style.backgroundColor = "rgba(0,0,0,0.85)";
    overlay.style.display = "flex";
    overlay.style.justifyContent = "center";
    overlay.style.alignItems = "center";
    overlay.style.zIndex = "99999";

    const modal = document.createElement("div");
    modal.style.backgroundColor = "black";
    modal.style.color = "red";
    modal.style.padding = "30px 50px";
    modal.style.borderRadius = "8px";
    modal.style.fontSize = "32px";
    modal.style.fontFamily = "Arial, sans-serif";
    modal.textContent = "YOU'VE BEEN HACKED 💀";

    overlay.appendChild(modal);
    document.body.appendChild(overlay);
})();
</scrIpt>
```


#### Successful Explotation

![](./images/XSS-No-WAF.png)



#### Brief Explanation 

The payload attack needed to be modified to bypass the Medium security filter cause in the previous security level there was no malitious script tratment. In medium level `<script>` and `</script>` keywords were translated into quotes but the validation was insensitive to upper and lower cases.

### Command Injection
#### Payload used
`192.168.0.0 & ls`

`192.168.0.0 & cat index.php`

#### Successful Explotation
The first payload was used to list the files in the directory.
![](./images/Command_injection1.png)

The second payload was used to view one of the files.
![](./images/Command_injection2.png)


#### Brief Explanation 
The application was vulnerable because it allowed use of the shell “&” operator. Although one might expect that at Medium security level such operators would be blocked, the filter seems to catch only && (and other typical separators), while & still passes — causing the shell to execute the injected command in parallel to the application’s ping. In consequence, this allowed execution of essentially any command on the server.

## **Part 3: Defensive Installation: Web Application Firewall (WAF)** 

### WAF Solution installed: Bunkerweb
### Detailed explanation of network setup
With docker and docker compose DVWA and Bunkerweb images were pulled from web. After that and with help of a intern docker network, Bunkerweb can serve as proxy of DVWA, mapping the host dvwa:80 (default port of dwva image) to the path /. Also, with the use of OWASP CRS, Bunkerweb protects DVWA from incomming potential attacks like SQL Injection.

Now, instead of port 4280 and URL http://localhost:4280/, DVWA is deployed with a WAF in front with URL http://localhost:8080/. All the requests first connect to the WAF to validate and allow the request and then go to the server DVWA wich can also connect and use the database server db.

![](./images/BunkerWeb-config.png)

### Defense Testing and Analysis
#### SQL Injection
For this attack the same input string and attack method (Postman and HTTP method) were used.

![](./images/SQL-Injection-WAF.png)

As its shown on the image, Bunkerweb succesfully blocked the SQL Injection attack showing a 403 status code on the page. Internally bunkerweb bunkerweb showed this message on its terminal indicating that the request matches with a SQLi vulnerabilitie:

```bash
 [NGINX.ERROR] 2025/11/28 00:39:26 [error] 384#384: *4233 [client 172.23.0.1] ModSecurity: Access denied with code 403 (phase 2). Matched "Operator `Ge' with parameter `5' against variable `TX:BLOCKING_INBOUND_ANOMALY_SCORE' (Value: `5' ) [file "/usr/share/bunkerweb/core/modsecurity/files/coreruleset-v4/rules/REQUEST-949-BLOCKING-EVALUATION.conf"] [line "222"] [id "949110"] [rev ""] [msg ""] [data ""] [severity "0"] [ver "OWASP_CRS/4.20.0"] [maturity "0"] [accuracy "0"] [tag "anomaly-evaluation"] [tag "OWASP_CRS"] [hostname "172.23.0.3"] [uri "/vulnerabilities/sqli/"] [unique_id "176429036646.967539"] [ref ""], client: 172.23.0.1, server: localhost, request: "POST /vulnerabilities/sqli/ HTTP/1.1", host: "localhost:8080"
```

#### XSS Reflected
For this attack the same input string and attack method (Web Browser) were used. The XSS injection is detected by Bunkerweb
![](./images/XSS-WAF.png)

Internally bunkerweb bunkerweb showed this message on its terminal indicating that the request matches with a XSS vulnerabilitie:

```bash

bunkerweb  | [NGINX.ERROR] 2025/11/28 04:43:54 [warn] 355#355: *1094 ModSecurity: Warning. detected XSS using libinjection. [file "/usr/share/bunkerweb/core/modsecurity/files/coreruleset-v4/rules/REQUEST-941-APPLICATION-ATTACK-XSS.conf"] [line "83"] [id "941100"] [rev ""] [msg "XSS Attack Detected via libinjection"] [data "Matched Data: XSS data found within ARGS:name: <br><sCript>(function(){const overlay = document.createElement(\x22div\x22);overlay.style.position = \x22fixed\x22;overlay.style.top = \x220\x22;overlay.style.left = \x220\x22;o (617 characters omitted)"] [severity "2"] [ver "OWASP_CRS/4.20.0"] [maturity "0"] [accuracy "0"] [tag "application-multi"] [tag "language-multi"] [tag "platform-multi"] [tag "attack-xss"] [tag "xss-perf-disable"] [tag "paranoia-level/1"] [tag "OWASP_CRS"] [tag "OWASP_CRS/ATTACK-XSS"] [tag "capec/1000/152/242"] [hostname "172.23.0.3"] [uri "/vulnerabilities/xss_r/"] [unique_id "17643050341.306744"] [ref "v33,772t:utf8toUnicode,t:urlDecodeUni,t:htmlEntityDecode,t:jsDecode,t:cssDecode,t:removeNulls"], client: 172.23.0.1, server: localhost
```

#### Command Injection
For this attack both payloads were tested: `192.168.0.0 & ls` and `192.168.0.0 & cat index.php`

![](./images/Command-injection-WAF.png)

As its shown on the image, Bunkerweb succesfully blocked the Command injection attack showing a 403 status code on the page. Internally bunkerweb bunkerweb showed this message on its terminal indicating that the request matches with a Remote Command Execution: Unix Command Injection (2-3 chars):
```bash
 [NGINX.ERROR] 2025/11/28 04:39:16 [warn] 372#372: *1327 ModSecurity: Warning. Matched "Operator `Rx' with parameter `(?i)(?:b[\"'\)\[\x5c]*(?:(?:(?:\|\||&&)[\s\x0b]*)?\$[!#\(\*\-0-9\?@_a-\{]*)?\x5c?u[\"'\)\[\x5c]*(?:(?:(?:\|\||&&)[\s\x0b]*)?\$[!#\(\*\-0-9\?@_a-\{]*)?\x5c?s[\"'\)\[\x5c]*(?:(?:(?:\|\||&&)[\s\x0b]*)?\$ (8328 characters omitted)' against variable `ARGS:ip' (Value: `192.168.0.0 & ls & cat index.php' ) [file "/usr/share/bunkerweb/core/modsecurity/files/coreruleset-v4/rules/REQUEST-932-APPLICATION-ATTACK-RCE.conf"] [line "122"] [id "932230"] [rev ""] [msg "Remote Command Execution: Unix Command Injection (2-3 chars)"] [data "Matched Data: & ls & cat index.php found within ARGS:ip: 192.168.0.0 & ls & cat index.php"] [severity "2"] [ver "OWASP_CRS/4.20.0"] [maturity "0"] [accuracy "0"] [tag "application-multi"] [tag "language-shell"] [tag "platform-unix"] [tag "attack-rce"] [tag "paranoia-level/1"] [tag "OWASP_CRS"] [tag "OWASP_CRS/ATTACK-RCE"] [tag "capec/1000/152/248/88"] [hostname "172.24.0.3"] [uri "/vulnerabilities/exec/"] [unique_id "176430475683.661376"] [ref "o12,20v939,32"], client: 172.24.0.1, server: localhost, request: "POST /vulnerabilities/exec/ HTTP/1.1", host: "localhost:8080", referrer: "http://localhost:8080/vulnerabilities/exec/"
```

#### Difference between an **Application Layer attack (L7)** you performed (like SQLi) and the **DDoS Application Layer attack** researched in Part 1 (like HTTP Flood)
The main difference between each type of attacks is the objective of the attack. In Application Layer attacks like SQLi hackers attacks webapps to steal data and informationto get an economic benefit in order to rescue the data. In the other hand, DDoS attacks main objective is to deny the service, endangering the communication of the server with internet and, in the case of entrepise applications and services, stopping all the production of the enterpise, asking for a economic benefit to stop the attack. Also, Application Layer attacks like SQLi take advantage of the infrastructure and communication between layers of the app to get sensitive or restricted information which shouldn't be accessed with actual roles and permission; for this reason, an important knowledge about frameworks, languages, OS and infraestructure of the app is useful to exploit some known vulnerabilities and access data. On the other side, DDoS attacks take advantage of the infraestructure limitations (CPU, RAM, bandwitch) to overcharge services, limiting the functionality; for this reason, a big computational resource is needed to overwhelm servers used in enterprises.





## References
https://coreruleset.org

https://owasp.org/Top10/A00_2021_Introduction/

https://www.cdnetworks.com/blog/cloud-security/types-of-ddos-attacks/

https://www.cloudflare.com/learning/ddos/what-is-a-ddos-attack/

https://www.imperva.com/learn/ddos/ddos-attacks/

https://www.cloudflare.com/learning/ddos/ntp-amplification-ddos-attack/

https://www.cloudflare.com/learning/ddos/ssdp-ddos-attack/

https://www.cloudflare.com/learning/ddos/what-is-an-ack-flood/

https://www.imperva.com/learn/ddos/ip-fragmentation-attack-teardrop/

https://www.imperva.com/learn/ddos/dns-flood/

https://cloud.google.com/blog/products/identity-security/how-it-works-the-novel-http2-rapid-reset-ddos-attack

https://www.imperva.com/learn/ddos/ntp-amplification/

https://blog.cloudflare.com/technical-details-behind-a-400gbps-ntp-amplification-ddos-attack/

https://www.cisa.gov/news-events/alerts/2014/01/13/ntp-amplification-attacks-using-cve-2013-5211
